In [ ]:
pip install WordCloud


In [ ]:
pip install textblob

In [21]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import logging
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

# Set up basic logging to output to the console
logging.basicConfig(level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# Define a function to authenticate user
def authenticate(token):
    valid_token = "secret_token"  # In practice, this should be stored securely
    if token == valid_token:
        return True
    else:
        logging.warning("Unauthorized access attempt.")
        return False

# Define a function to clean text data
def clean_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.strip()  # Remove extra whitespace
    return text

# Create a synthetic dataset
np.random.seed(0)
reviews = [
    "I love this product! It has changed my life for the better.",
    "Absolutely terrible. I would never recommend this to anyone.",
    "Pretty decent, but could use some improvements.",
    "An excellent choice, very pleased with the purchase.",
    "Not what I expected, and it broke after a week.",
    "Fantastic! Does exactly what it says it will.",
    "I'm not happy with this product at all.",
    "The best purchase I've made in years!",
    "Amazing, I loved this incredible, great product!!",
    "Not good, not helpfull, bad, and a waste of time. Save your money.",
    "The innovation in this product is remarkable, leading to an outstanding user experience.",
    "Spectacular performance! It has consistently outperformed my expectations.",
    "What a delightful find! It's been a joy to use, and the results are impressive.",
    "The durability and reliability of this product are unmatched. Highly recommended!",
    "It's rare to find a product that delivers on all fronts – this one truly does.",
    "The design is flawed, making it difficult to use effectively. Very disappointed.",
    "After just a few uses, it started malfunctioning. Not what I signed up for.",
    "The customer service was as unsatisfactory as the product itself. Won't be back.",
    "It's just not worth the investment. There are better options available on the market.",
    "I had high hopes, but unfortunately, the product failed to deliver on its promises."

    # Add more reviews as needed
]

# Manually set sentiments for the reviews to ensure accuracy
sentiments = [
    1,  # Positive
    0,  # Negative
    0,  # Negative
    1,  # Positive
    0,  # Negative
    1,  # Positive
    0,  # Negative
    1,  # Positive
    1,  # Positive
    0,  # Negative
    1,  # Positive
    1,  # Positive
    1,  # Positive
    1,  # Positive
    1,  # Positive
    0,  # Negative
    0,  # Negative
    0,  # Negative
    0,  # Negative
    0,  # Negative

    # Add more sentiments (1 for positive, 0 for negative) corresponding to each review
]

# Create a DataFrame
df = pd.DataFrame({'Review': reviews, 'Sentiment': sentiments})

# Apply the cleaning function to the review column and split the dataset
df['Cleaned_Review'] = df['Review'].apply(clean_text)
X_train, X_test, y_train, y_test = train_test_split(df['Cleaned_Review'], df['Sentiment'], test_size=0.2, random_state=42)

# Create a text processing and model pipeline
model_pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Train the model
model_pipeline.fit(X_train, y_train)

# Evaluate the model's performance on the test set
y_pred = model_pipeline.predict(X_test)
logging.info("\n" + classification_report(y_test, y_pred))

# Visualization #1: Bar Chart of Sentiment Distribution
plt.figure(figsize=(8, 6))  # Set the figure size
sns.countplot(x='Sentiment', data=df)
plt.title('Sentiment Distribution')
plt.ylabel('Count')
plt.xlabel('Sentiment')
plt.xticks([0, 1], ['Negative', 'Positive'])  # Set custom x-axis labels
plt.show()

# Visualization #2: Pie Chart of Sentiment Distribution
plt.figure(figsize=(8, 6))  # Set the figure size
df['Sentiment'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=140, labels=['Positive', 'Negative'])
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Sentiment Distribution')
plt.show()

# Visualization #3: Word Cloud
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(' '.join(df['Cleaned_Review']))
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Interactive Prediction Function
def predict_sentiment(review, token):
    if not authenticate(token):
        print("Unauthorized. Please provide a valid token.")
        return
    cleaned_review = clean_text(review)
    prediction = model_pipeline.predict([cleaned_review])
    sentiment = "Positive" if prediction[0] == 1 else "Negative"
    logging.info(f"Sentiment predicted: {sentiment} for review: '{review}'")
    print(f"The predicted sentiment for the review is: {sentiment}")

# Interactive Widgets
review_input = widgets.Textarea(placeholder='Type your review here...', description='Review:', layout={'width': '600px', 'height': '100px'})
token_input = widgets.Text(placeholder='Enter your token here...', description='Token:')
predict_button = widgets.Button(description="Predict Sentiment")

def on_predict_button_clicked(b):
    clear_output(wait=True)
    display(review_input, token_input, predict_button)
    predict_sentiment(review_input.value, token_input.value)

predict_button.on_click(on_predict_button_clicked)
display(review_input, token_input, predict_button)


Textarea(value='Amazing! I love this', description='Review:', layout=Layout(height='100px', width='600px'), pl…

Text(value='secret_token', description='Token:', placeholder='Enter your token here...')

Button(description='Predict Sentiment', style=ButtonStyle())

2023-11-03 11:38:44,612:INFO:Sentiment predicted: Positive for review: 'Amazing! I love this'


The predicted sentiment for the review is: Positive
